# データの整形

先ほどの学生実験用アンケート結果をダウンロードして Pandas に読み込んでみましょう。

* <a href="https://docs.google.com/spreadsheets/d/1u50kS8Ztmgjjs--S1AM4753quL3QEPRR7xIGjCxcSCw/edit#gid=941271295" target="_blank">学生実験用アンケート結果</a>

    * 左上の「File」→「Download as」→「Comma-separated values (.csv, current sheet)」で保存
    * 保存時の名前は「Questionary - Form Responses 1.csv」のままにしてください。
    * 保存したファイルを、現在実行中の .ipynb ファイルと同じ場所に置く（移動させる）。

In [ ]:
# データフレーム操作に関するライブラリをインポートする
import pandas as pd

In [ ]:
# URL によるリソースへのアクセスを提供するライブラリをインポートする。
# import urllib # Python 2 の場合
import urllib.request # Python 3 の場合

In [ ]:
# データの読み込み
df1 = pd.read_csv('./Questionary - Form Responses 1.csv', sep=',') 

In [ ]:
df1

今回は、この書式のままだと非常に使いにくいので、使いやすい形に整形します。以下のコードを実行してください（このコードの詳細は、今は理解しなくても大丈夫です）。

In [ ]:
import re
seen = []
output = open('./Responses.csv', 'w')
with open('./Questionary - Form Responses 1.csv') as f:
    for i, line in enumerate(f.readlines()):
        if i == 0:
            line = re.sub(r'\..*?\,', ',', line)
            line = re.sub(r'\..*?$', '', line)
            a = line.split(",")
            output.write(",".join([str(w) for w in a[2:]]))
        else:
            a = line.split(",")
            if a[0] == '':
                continue
            for j in range(len(a)):
                if len(seen) < j + 1:
                    seen.append([])
                if a[j] not in seen[j]:
                    seen[j].append(a[j])
                #print(j, a[j])
            for j in [3, 11, 12, 13, 20]:
                a[j] = seen[j].index(a[j])
            output.write(",".join([str(w) for w in a[2:]]))
output.close()

これで、整形後のデータ Responses.csv が得られて、以降の解析に使えるはずです。

In [ ]:
# データの読み込み
df2 = pd.read_csv('Responses.csv', sep=',') 

In [ ]:
df2

これでうまく読み込めました。

In [ ]:
# 明らかに変だろという行を指定して取り除きたいときの例
df2.drop(23)

In [ ]:
# でも、１個１個データを見て取り除いていくのは辛いですね。自動的に取り除く方法を作ってみましょう。

In [ ]:
#ちなみに、質問項目を改めて出力しておくと、以下のようになります。
pd.DataFrame(df1.columns[2:])

In [ ]:
# 標準偏差と平均値で標準化して、標準化した値（Z値）が閾値以上の答えを外れ値と見なし、外れ値を答えた人を除外する。
def remove_outliers(df, col_name, threshold):
    dfs = df[col_name].apply(lambda x: (x-df[col_name].mean())/df[col_name].std())
    return df[dfs < threshold]

In [ ]:
# 標準偏差と平均値で標準化して、標準化した値（Z値）が閾値以上の答えを外れ値と見なし、外れ値を答えた人を除外する。
df2 = remove_outliers(df2, 'Q2', 2)
df2 = remove_outliers(df2, 'Q3', 2)
df2 = remove_outliers(df2, 'Q4', 2)
df2 = remove_outliers(df2, 'Q5', 2)
df2 = remove_outliers(df2, 'Q6', 2)
df2 = remove_outliers(df2, 'Q12', 2)
df2 = remove_outliers(df2, 'Q13', 2)
df2 = remove_outliers(df2, 'Q14', 2)
df2 = remove_outliers(df2, 'Q15', 2)
df2 = remove_outliers(df2, 'Q16', 2)
df2 = remove_outliers(df2, 'Q17', 2)

In [ ]:
df2

In [ ]:
# Q18 は「彼氏または彼女はいる？」 0: いる（リアル）、 1: いる（２次元）、 2:いない だが、
# 「1: いる（２次元）」と「2:いない」をどちらも同じと見なして「1」としたい場合。
# （文法は難しいので、今回は見よう見まねで使ってください）
df2['Q18'] = [1 if i > 0 else 0 for i in df2['Q18'].tolist()]

In [ ]:
df2

In [ ]:
# Q19 あなたはリア充ですか？ は５段階評価だが、
# ３より上を「1.リア充」、３以下を「0. リア充ではない」の２段階評価に変換したい場合
# （文法は難しいので、今回は見よう見まねで使ってください）
df2['Q19'] = [1 if i > 3 else 0 for i in df2['Q19'].tolist()]

In [ ]:
df2